# Quarterly Census of Employment and Wages

###### Imports

In [2]:
import pandas as pd
import numpy as np
import requests

from __future__ import print_function, division
import matplotlib.pylab as pl
import seaborn as sns
sns.set_style('whitegrid')
# import json

# Spatial
import geopandas as gpd
import fiona
import shapely

import urllib.request
import urllib

#import statsmodels.formula.api as smf
#import statsmodels.api as sm

%pylab inline

Populating the interactive namespace from numpy and matplotlib


###### Functions for Gathering Data  
This code is from the Bureau of Labor Statistics sample functions Python 3.x Example: https://data.bls.gov/cew/doc/access/data_access_examples.htm#PYTHON

In [3]:
# *******************************************************************************
# qcewCreateDataRows : This function takes a raw csv string and splits it into
# a two-dimensional array containing the data and the header row of the csv file
# a try/except block is used to handle for both binary and char encoding
def qcewCreateDataRows(csv):
    dataRows = []
    try: dataLines = csv.decode().split('\r\n')
    except er: dataLines = csv.split('\r\n');
    for row in dataLines:
        dataRows.append(row.split(','))
    return dataRows
# *******************************************************************************


# *******************************************************************************
# qcewGetAreaData : This function takes a year, quarter, and area argument and
# returns an array containing the associated area data. Use 'a' for annual
# averages. 
# For all area codes and titles see:
# http://www.bls.gov/cew/doc/titles/area/area_titles.htm
#
def qcewGetAreaData(year,qtr,area):
    urlPath = "http://data.bls.gov/cew/data/api/[YEAR]/[QTR]/area/[AREA].csv"
    urlPath = urlPath.replace("[YEAR]",year)
    urlPath = urlPath.replace("[QTR]",qtr.lower())
    urlPath = urlPath.replace("[AREA]",area.upper())
    httpStream = urllib.request.urlopen(urlPath)
    csv = httpStream.read()
    httpStream.close()
    return qcewCreateDataRows(csv)
# *******************************************************************************


# *******************************************************************************
# qcewGetIndustryData : This function takes a year, quarter, and industry code
# and returns an array containing the associated industry data. Use 'a' for 
# annual averages. Some industry codes contain hyphens. The CSV files use
# underscores instead of hyphens. So 31-33 becomes 31_33. 
# For all industry codes and titles see:
# http://www.bls.gov/cew/doc/titles/industry/industry_titles.htm
#
def qcewGetIndustryData(year,qtr,industry):
    urlPath = "http://data.bls.gov/cew/data/api/[YEAR]/[QTR]/industry/[IND].csv"
    urlPath = urlPath.replace("[YEAR]",year)
    urlPath = urlPath.replace("[QTR]",qtr.lower())
    urlPath = urlPath.replace("[IND]",industry)
    httpStream = urllib.request.urlopen(urlPath)
    csv = httpStream.read()
    httpStream.close()
    return qcewCreateDataRows(csv)
# *******************************************************************************


# *******************************************************************************
# qcewGetSizeData : This function takes a year and establishment size class code
# and returns an array containing the associated size data. Size data
# is only available for the first quarter of each year.
# For all establishment size classes and titles see:
# http://www.bls.gov/cew/doc/titles/size/size_titles.htm
#
def qcewGetSizeData(year,size):
    urlPath = "http://data.bls.gov/cew/data/api/[YEAR]/1/size/[SIZE].csv"
    urlPath = urlPath.replace("[YEAR]",year)
    urlPath = urlPath.replace("[SIZE]",size)
    httpStream = urllib.request.urlopen(urlPath)
    csv = httpStream.read()
    httpStream.close()
    return qcewCreateDataRows(csv)

In [4]:
#make some lists to try "get "

# def qcewGetAreaData(year,qtr,area):
#     urlPath = "http://data.bls.gov/cew/data/api/[YEAR]/[QTR]/area/[AREA].csv"
#     urlPath = urlPath.replace("[YEAR]",year)
#     urlPath = urlPath.replace("[QTR]",qtr.lower())
#     urlPath = urlPath.replace("[AREA]",area.upper())
#     httpStream = urllib.request.urlopen(urlPath)
#     csv = httpStream.read()
#     httpStream.close()
#     return qcewCreateDataRows(csv)

test = qcewGetAreaData('2020', 'A', '47189')
test = pd.DataFrame(test) # put it in a pandas table
test.columns = test.iloc[0] # first row to headers
test = test[1:] # same
#test.columns = [i.replace('"', '') for i in test.columns] # cleaning data
#test = test.replace({'"':''}, regex=True) # cleaning data

In [6]:
test.head(30)

,"""area_fips""","""own_code""","""industry_code""","""agglvl_code""","""size_code""","""year""","""qtr""","""disclosure_code""","""annual_avg_estabs""","""annual_avg_emplvl""",...,"""oty_total_annual_wages_chg""","""oty_total_annual_wages_pct_chg""","""oty_taxable_annual_wages_chg""","""oty_taxable_annual_wages_pct_chg""","""oty_annual_contributions_chg""","""oty_annual_contributions_pct_chg""","""oty_annual_avg_wkly_wage_chg""","""oty_annual_avg_wkly_wage_pct_chg""","""oty_avg_annual_pay_chg""","""oty_avg_annual_pay_pct_chg"""
1,"""47189""","""0""","""10""","""70""","""0""","""2020""","""A""","""""",3285,50782,...,245322898,11.7,36251798,8.8,92407,2.4,57,6.9,2944,6.8
2,"""47189""","""1""","""10""","""71""","""0""","""2020""","""A""","""""",17,238,...,834041,5.9,0,0.0,0,0.0,35,3.0,1806,3.0
3,"""47189""","""1""","""102""","""72""","""0""","""2020""","""A""","""""",17,238,...,834041,5.9,0,0.0,0,0.0,35,3.0,1806,3.0
4,"""47189""","""1""","""1021""","""73""","""0""","""2020""","""A""","""""",7,181,...,300471,2.9,0,0.0,0,0.0,70,6.5,3636,6.5
5,"""47189""","""1""","""1028""","""73""","""0""","""2020""","""A""","""""",10,57,...,533570,15.2,0,0.0,0,0.0,-166,-10.9,-8635,-10.9
6,"""47189""","""1""","""22""","""74""","""0""","""2020""","""A""","""""",1,9,...,-28467,-2.7,0,0.0,0,0.0,209,10.6,10839,10.6
7,"""47189""","""1""","""221""","""75""","""0""","""2020""","""A""","""""",1,9,...,-28467,-2.7,0,0.0,0,0.0,209,10.6,10839,10.6
8,"""47189""","""1""","""2211""","""76""","""0""","""2020""","""A""","""""",1,9,...,-28467,-2.7,0,0.0,0,0.0,209,10.6,10839,10.6
9,"""47189""","""1""","""22111""","""77""","""0""","""2020""","""A""","""""",1,9,...,-28467,-2.7,0,0.0,0,0.0,209,10.6,10839,10.6
10,"""47189""","""1""","""221111""","""78""","""0""","""2020""","""A""","""""",1,9,...,-28467,-2.7,0,0.0,0,0.0,209,10.6,10839,10.6


In [ ]:
industry = {'office':['1022', '1023', '1024'],
            'health':['1025']}
            # continue adding industries as needed
            # in the format of
            # '<IndustryName>' : [<list of ind_codes as str>] 
    
ownership = '5' # could become a list if you want to include more ownership codes

In [ ]:
list(industry.keys()) # dictionaries are alphabething themselves!

In [ ]:
# TEST ONE COUNTY
# this one is a test of one STCO -- will not be stored

test = qcewGetAreaData("2017","A",'06041') # using BLS code (only after 2013!)
test = pd.DataFrame(test) # put it in a pandas table
test.columns = test.iloc[0] # first row to headers
test = test[1:] # same
test.columns = [i.replace('"', '') for i in test.columns] # cleaning data
test = test.replace({'"':''}, regex=True) # cleaning data
test = test[['area_fips', 'own_code', 'industry_code', 'annual_avg_emplvl']] # selecting only relevant columns

# index to relevant row by ownership and industry
test = test.loc[(test['own_code'] == ownership) & test['industry_code'].isin(industry['office'])] 

# summing all rows to total and create new row
test.annual_avg_emplvl = test.annual_avg_emplvl.astype(int)
test = test.append(test.sum(numeric_only=True), ignore_index=True)

# assigning fipa, own, industry data to the new row
test['area_fips'][-1:] = test['area_fips'][:1]
test['own_code'][-1:] = test['own_code'][:1]
test['industry_code'][-1:] = 'office'

test = test[-1:] # dropping all rows but the sum
test.annual_avg_emplvl = test.annual_avg_emplvl.astype(int)

print(test.shape)
print(test.dtypes)
test.head()